# **8 Puzzle**

### 8 Puzzle is a sliding puzzle.

## Libraries Used:
- queue
- time
- os
- psutil
- sys
- copy
- pydot
- graphviz




In [1]:
!pip install pydot
!pip install graphviz
import pydot
import graphviz

In [3]:
from helperfunctions import *

Goal matrix looks like this:



In [4]:
goal_matrix = [
  [1, 2, 3],
  [4, 5, 6],
  [7, 0, 8]
]

print("Goal state:")
print_matrix(goal_matrix)

Goal state:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]


The 0 tile is the only tile that can be slid

**Input Method 1:** Use input files that are provided in the input folder. Feel free to switch between input files by changing the value of `input_file`.

In [5]:
input_file = 'input.txt'

In [6]:
f = open(input_file)
line = f.readline()
results = line.split(', ')
arr = [int(i) for i in results]
f.close()

matrix = get_matrix(arr)  
print("Initial state:")
print_matrix(matrix)

Initial state:
[1, 2, 3]
[4, 5, 0]
[8, 6, 7]


**Input Method 2:** Take user input row wise.

In [ ]:
def get_user_input_matrix():
  arr =[]
  print("Enter the entries row wise: (Press enter after each entry)") 
  for i in range(9):
    if i % 3 == 0:
      print("row " + str(int(i/3)+1) + ": ")
    value = int(input())
    while value in arr or value not in range(9):
      print("Invalid input '" + str(value) + "' is detected. Please input again.")
      value = int(input())
    arr.append(value)
  return get_matrix(arr)

In [ ]:
matrix = get_user_input_matrix()
#ensure the user-inputted matrix is not identical to goal_matrix
while matrix == goal_matrix:
  print("\nInitial state inputted is identical to the goal state. The initial state and goal state should not be the same.")
  print("Please input the whole initial state again.")
  matrix = get_user_input_matrix()

print("\nInitial state:")
print_matrix(matrix)

Enter the entries row wise: (Press enter after each entry)
row 1: 
1
2
3
row 2: 
4
5
0
row 3: 
8
6
7

Initial state:
[1, 2, 3]
[4, 5, 0]
[8, 6, 7]


**Input Method 3:** Use random number generated as input.

In [ ]:
from random import seed
from random import randint
from datetime import datetime

def get_random_matrix():
  seed(datetime.now())
  arr = []
  for i in range(9):
    value = randint(0, 8)
    while value in arr:
      value = randint(0, 8)
    arr.append(value)
    
  return get_matrix(arr)

In [ ]:
matrix = get_random_matrix()
#ensure the random generated matrix is not identical to goal_matrix
while matrix == goal_matrix:  
  matrix = get_random_matrix()

print("Initial state:")
print_matrix(matrix)

Initial state:
[4, 2, 8]
[7, 5, 6]
[1, 0, 3]



Node class for states of puzzle

Contents:
 - matrix: state of puzzle
 - parent: state of puzzle that the node is derived from
 - move: move made by parent node to get to state
 - g_n + h_n = f_n


## H1 & H2

* h1 = total number of tiles that are in wrong row + total number of tiles that 
are in wrong column
* h2 = summation of Euclidean distances of all the tiles 

In [8]:
import math
"""
  Function for search heuristic 1 (searching)
"""
def h1Search(arr, x):  
    for i in range(len(arr)): 
        if arr[i] == x: 
            return True
    return False
"""
  Function for calculating heuristic 1 (misplaced_tiles)
"""
def h1_misplaced_tiles(matrix):
  misplaced_tiles = 0
  misplaced_rowTiles = 0
  misplaced_colTiles = 0
  i, j = 0, 0
  k, l = 0, 0
  rowTilesSet = [[goal_matrix[0][0],goal_matrix[0][1],goal_matrix[0][2]], 
                 [goal_matrix[1][0],goal_matrix[1][1],goal_matrix[1][2]],
                 [goal_matrix[2][0],goal_matrix[2][1],goal_matrix[2][2]]]
  colTilesSet = [[goal_matrix[0][0],goal_matrix[1][0],goal_matrix[2][0]], 
                 [goal_matrix[0][1],goal_matrix[1][1],goal_matrix[2][1]],
                 [goal_matrix[0][2],goal_matrix[1][2],goal_matrix[2][2]]]
  rowMatrix = [[matrix[0][0],matrix[0][1],matrix[0][2]], [matrix[1][0],matrix[1][1],matrix[1][2]], [matrix[2][0],matrix[2][1],matrix[2][2]]]
  colMatrix = [[matrix[0][0],matrix[1][0],matrix[2][0]], [matrix[0][1],matrix[1][1],matrix[2][1]], [matrix[0][2],matrix[1][2],matrix[2][2]]]
  while (i < 3):
    j = 0
    while (j < 3):
      if (not(h1Search(rowTilesSet[i],(rowMatrix[i])[j])) and (rowMatrix[i])[j]!= 0):
        misplaced_rowTiles = misplaced_rowTiles + 1
      if (not(h1Search(colTilesSet[i],(colMatrix[i])[j])) and (colMatrix[i])[j]!= 0):
        misplaced_colTiles = misplaced_colTiles + 1
      j = j + 1
    i = i + 1
  misplaced_tiles = misplaced_rowTiles + misplaced_colTiles
  return misplaced_tiles  


"""
  Function for calculating heuristic 2 (euclidean distance)
"""
def h2_euclidean_distance(matrix):
  i = 0
  distance = 0
  while (i < 3):
    j = 0
    while (j < 3):
      if(matrix[i][j] != goal_matrix[i][j] and matrix[i][j] != 0):
        value = matrix[i][j]
        k = 0
        while (k < 3):
          l = 0
          while (l < 3):
            if (goal_matrix[k][l] == value):
              dis = math.sqrt(((k - i)**2) + ((l - j)**2))
              distance = distance + dis
              l, k = 3, 3
            l = l + 1
          k = k + 1
      j = j + 1
    i = i + 1
  return distance

## Search Tree Solution (Drawing)

In [9]:
class Solution(object):
  def __init__(self,cutoff):
      self.graph = pydot.Dot(graph_type="digraph", bgcolor="#fff3af", strict=True, label="Cutoff: " + str(cutoff),
                            labelloc="top", labeljust="left", fontcolor="black", fontsize="40")
      self.visited = dict()
      self.goal = None

  def write_image(self, file_name: str = "out.png") -> object:
      """
      :param file_name: Name of the output file to be written
      """
      try:
          self.graph.write_png(file_name)
          print(f"Image {file_name} written successfully.")
      except Exception as e:
          print("Error Writing image", e)

## IDA*

In [10]:
"""
  Function to solve 8 puzzle with Iterative Deepening A*
  Arguments:
    matrix: 3 by 3, list of 3 lists
"""
def idastar(matrix, heuristic):
  
  initial_time = time.process_time()
  process = psutil.Process(os.getpid())
  initial_memory = process.memory_info().rss / 1024.000000
  
  # initialization
  path = []
  cutoff = 0
  expanded_nodes = []
  global explored_nodes
  explored_nodes = []
  global finalIterationH1
  global finalIterationH2

  # set cutoff
  if (heuristic == 'h1'):
    cutoff = h1_misplaced_tiles(matrix)
  else:
    cutoff = h2_euclidean_distance(matrix)

  root = Node(matrix)
  path.insert(0, root)

  iterations = 0

  while True:
    explored_nodes = []

    if (iterations!=0):
      i.graph.write_png(heuristic+"Iteration"+str(iterations)+".png") 
    iterations += 1
    i = Solution(cutoff)

    print ("Iteration = " , iterations)
    print ("Cutoff = " , cutoff)
    
    # begin searching
    temp = search(path, 0, cutoff, heuristic,iterations,i)

    if (temp == 'found'):
      print('\nSolved in', iterations, 'iterations using', heuristic)
      elapsed_time = time.process_time() - initial_time
      final_memory = process.memory_info().rss / 1024.000000
      memory_used = final_memory - initial_memory
      print_search_info(path[0], len(explored_nodes), elapsed_time, memory_used)
      i.graph.write_png(heuristic+"Iteration"+str(iterations)+".png") 
      if (heuristic == 'h1'):
        finalIterationH1 = iterations
      else:
        finalIterationH2 = iterations
      return path[0]

    # no solution
    if (temp == float('inf')):
      print('This algorithm was not able to find a solution')
      return

    # increase cutoff
    cutoff = temp

In [11]:
"""
  Helper function to idastar().
  Arguments:
    path: nodes being searched
    g: path cost
    cutoff: current threshold
  Returns:
    'found' if solution is found
    new cutoff(int) if higher cutoff is needed
    float('inf') if no solution
"""
def search(path, g, cutoff, heuristic,iterations,i): 
  node = path[0]
  
  explored_nodes.append(node.matrix)
 
  if (heuristic == 'h1'):
    h = h1_misplaced_tiles(node.matrix)
    f = g + h1_misplaced_tiles(node.matrix)
  else:
    h = h2_euclidean_distance(node.matrix)
    f = g + h2_euclidean_distance(node.matrix)

  if (node.matrix != goal_matrix):
    nodeColor = "lightgray"
  else:
    nodeColor = "green"
  
  n = pydot.Node(str(node.matrix),style="filled",fillcolor=nodeColor)
  i.graph.add_node(n)
  if node.parent != None:
    edge = pydot.Edge(str(node.parent.matrix), str(node.matrix),label=node.move+ "\n g: " + str(g) + "\n h: " + str(h) + "\n f: " + str(f))
    i.graph.add_edge(edge)

  #print_matrix(node.matrix)
  #print ("Gn = ", g)

  if (f > cutoff):
    return f  # greater f found

  if (node.matrix == goal_matrix):
    return 'found' 

  min = float('inf')

  for child in get_children(node, []):
    # node already explored or in path, so skip it
    if child.matrix in explored_nodes:
      continue
    if child in path:
      continue

    # if child is not None, search recursively
    if (child.matrix != None):
      path.insert(0, child) 
      temp = search(path, g + 1, cutoff, heuristic,iterations,i)

      if (temp == 'found'):
        # solution found
        return 'found'

      if (temp < float('inf')):
        # higher cutoff found
        min = temp

      path.pop(0)

  # no solution, return infinity
  return min

## Main (H1)

In [12]:
from IPython.display import Image
def show_png(iterations, heuristic):
    return Image(filename= heuristic+"Iteration"+str(iterations)+".png", width=1000)

In [13]:
print('8 Puzzle using Iterative Deepening A*')
node = idastar(matrix, 'h1')

8 Puzzle using Iterative Deepening A*
Iteration =  1
Cutoff =  4
Iteration =  2
Cutoff =  6
Iteration =  3
Cutoff =  8
Iteration =  4
Cutoff =  10
Iteration =  5
Cutoff =  12

Solved in 5 iterations using h1

Moves:  ['U', 'R', 'R', 'D', 'L', 'L', 'U', 'R', 'D', 'R', 'U', 'L']
Number of Nodes  Expanded:  98
Time Taken:  0.2370176 nanoseconds
Memory Used:  4916.00 KB



In [14]:
from ipywidgets import interact, widgets, fixed
interact(show_png, iterations=widgets.IntSlider(min=1,max=finalIterationH1,step=1,value=1), heuristic=fixed('h1'));

interactive(children=(IntSlider(value=1, description='iterations', max=5, min=1), Output()), _dom_classes=('wi…

## Main (H2)

In [15]:
print('8 Puzzle using Iterative Deepening A*')
node = idastar(matrix, 'h2')

8 Puzzle using Iterative Deepening A*
Iteration =  1
Cutoff =  5.414213562373095
Iteration =  2
Cutoff =  6.650281539872885
Iteration =  3
Cutoff =  7.23606797749979
Iteration =  4
Cutoff =  9.23606797749979
Iteration =  5
Cutoff =  11.23606797749979
Iteration =  6
Cutoff =  13.23606797749979

Solved in 6 iterations using h2

Moves:  ['U', 'R', 'R', 'D', 'L', 'L', 'U', 'R', 'D', 'R', 'U', 'L']
Number of Nodes  Expanded:  84
Time Taken:  0.2685884 nanoseconds
Memory Used:  300.00 KB



In [16]:
interact(show_png, iterations=widgets.IntSlider(min=1,max=finalIterationH2,step=1,value=1), heuristic=fixed('h2'));

interactive(children=(IntSlider(value=1, description='iterations', max=6, min=1), Output()), _dom_classes=('wi…